In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [9]:
import torch
from transformers import TrainingArguments, Trainer
from unsloth import FastVisionModel
from datasets import load_dataset, Dataset
import pandas as pd
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import json
import os
from io import BytesIO
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import TextStreamer
import string
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [3]:

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.4.7: Fast Mllama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [4]:
# Apply LoRA fine-tuning setup
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    random_state=42,
    use_rslora=True,
)

Unsloth: Making `model.base_model.model.vision_model.transformer` require gradients


In [5]:
# 1. 讀取 parquet 成 pandas DataFrame（保留 image 為 bytes）
train_df = pd.read_parquet("train_data.parquet")
valid_df = pd.read_parquet("valid_data.parquet")

# 2. 轉成 HuggingFace Dataset（此時 image 還是 bytes，不會報錯）
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# 3. 轉成 conversation 格式（這裡再轉成 PIL）
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

def convert_bytes_to_pil(byte_data):
    return Image.open(io.BytesIO(byte_data)).convert("RGB")

def convert_to_conversation(sample):
    pil_image = convert_bytes_to_pil(sample["image"]["bytes"])
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : pil_image } ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["caption"]} ]
        },
    ]
    return { "messages" : conversation }

# 4. 使用 Dataset.map() 處理成 conversation 格式（延後處理 PIL）
train_dataset = [convert_to_conversation(sample) for sample in train_dataset]
valid_dataset = [convert_to_conversation(sample) for sample in valid_dataset]

In [6]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from unsloth import is_bf16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    args = SFTConfig(
        output_dir = "outputs",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        max_steps = 30,
        warmup_steps = 10,
        learning_rate = 2e-4,
        weight_decay = 0.01,
        logging_steps = 1,
        seed = 3407,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        optim = "adamw_8bit",
        lr_scheduler_type = "cosine",
        report_to = "none",

        # Vision tuning 必要設定
        remove_unused_columns = False,
        dataset_text_field = "",  # 留空即可，讓 unsloth 自動用 messages
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    )
)


In [8]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 134,348,800/11,000,000,000 (1.22% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.610800
2,4.534400
3,4.432000
4,3.967700
5,3.376100
6,2.908300
7,2.546200
8,2.132200
9,1.582100
10,1.380600


TrainOutput(global_step=30, training_loss=1.66600807706515, metrics={'train_runtime': 4747.0441, 'train_samples_per_second': 0.101, 'train_steps_per_second': 0.006, 'total_flos': 1427440484381472.0, 'train_loss': 1.66600807706515})

**INFERENCE**

In [ ]:

bleu_scores = []
references = []
hypotheses = []

smooth = SmoothingFunction().method1

def extract_assistant_response(text):
    # 擷取 <|assistant|> 後段
    if "<|assistant|>" in text:
        text = text.split("<|assistant|>")[-1]

    # 移除所有 <|...|> 標記
    text = re.sub(r"<\|.*?\|>", "", text)

    # 移除開頭可能重複出現的 user prompt
    prompt = "You are an expert radiographer. Describe accurately what you see in this image."
    text = text.replace("user", "")  # 移除孤立的 "user"
    text = text.replace("assistant", "")
    text = text.replace(prompt, "")  # 移除 prompt 本身
    text = re.sub(r"\n+", " ", text)
    return text.strip()

def normalize_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip()

for i, example in enumerate(tqdm(valid_dataset, desc="Validating")):
    try:
        # ✅ 擷取 Ground Truth
        messages = example["messages"]
        user_msg = messages[0]["content"]
        gt_response = messages[1]["content"][0]["text"].strip()
        gt_normalized = normalize_text(gt_response)

        # ✅ 擷取圖與指令
        image = None
        instruction = ""
        for item in user_msg:
            if item["type"] == "image":
                image = item["image"]
            elif item["type"] == "text":
                instruction = item["text"]

        assert image is not None, f"[{i}] ❌ 圖片遺失"
        assert instruction.strip() != "", f"[{i}] ❌ 指令為空"

        # ✅ 构造 messages + 編碼
        messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": instruction}]}]
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(image, input_text, add_special_tokens=False, return_tensors="pt").to("cuda")
        text_streamer = TextStreamer(tokenizer, skip_prompt=True)

        # ✅ 推理
        with torch.no_grad():
            output = model.generate(
                **inputs,
                streamer=text_streamer,
                max_new_tokens=128,
                use_cache=True,
                temperature=1.5,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=False).strip()
        gen_text = extract_assistant_response(decoded)
        gen_normalized = normalize_text(gen_text)

        print(f"\n📌 GT: {gt_response}")
        print(f"🤖 OUT: {gen_text}")

        # ✅ 單句 BLEU
        score = sentence_bleu([gt_normalized.split()], gen_normalized.split(), smoothing_function=smooth)
        print(f"🎯 BLEU: {score:.4f}")
        bleu_scores.append(score)

        # ✅ 收集資料以供 corpus BLEU
        references.append([gt_normalized.split()])
        hypotheses.append(gen_normalized.split())

    except Exception as e:
        print(f"\n⚠️ [Example {i+1}] 發生錯誤：{repr(e)}\n")
        continue

# ✅ 顯示平均 BLEU 分數
if bleu_scores:
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"\n✅ 平均 BLEU 分數: {avg_bleu:.4f}")
else:
    print("❌ 無可評估項目")

# ✅ 額外計算整體 Corpus BLEU
if references and hypotheses:
    corpus_bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smooth)
    print(f"📊 Corpus BLEU 分數（整體）: {corpus_bleu_score:.4f}")
else:
    print("❌ 無法計算 Corpus BLEU")

Validating:   0%|          | 0/100 [00:00<?, ?it/s]

Christmast Cat Art - Old Art for Christmas 1909, vintage greeting card of cat orchestra - Christmas vintage design<|eot_id|>


Validating:   1%|          | 1/100 [00:06<11:20,  6.87s/it]


📌 GT: To Wish You A Jolly Christmas: To Wish You A Jolly Christmas Cat Cow Dog Fiddle Music Artist Unknown Year ca 1910 Public Domain
🤖 OUT: Christmast Cat Art - Old Art for Christmas 1909, vintage greeting card of cat orchestra - Christmas vintage design
🎯 BLEU: 0.0100
Knitting 12 - Knitting pattern on canvas<|eot_id|>


Validating:   2%|▏         | 2/100 [00:11<08:38,  5.29s/it]

...


📌 GT: Fairy Hat On Plant: Tiny inverted plant part that looks like the hat for a fairy
🤖 OUT: A Tiny Crab: Little brown crab on tree with water drops
🎯 BLEU: 0.0157
White-bellied Whistling Duck: White-bellied whistling duck in closeup on dark background<|eot_id|>


Validating:  40%|████      | 40/100 [04:01<05:58,  5.98s/it]


📌 GT: Keen Eye Of Spurwinged Goose: Close view of keen eye of spurwinged goose
🤖 OUT: White-bellied Whistling Duck: White-bellied whistling duck in closeup on dark background
🎯 BLEU: 0.0000
Gray Surface: Gray wall<|eot_id|>


Validating:  41%|████      | 41/100 [04:05<05:07,  5.20s/it]


📌 GT: Gray Gradient Natural Background: Gray fog color gradient in natural occurring event
🤖 OUT: Gray Surface: Gray wall
🎯 BLEU: 0.0129
Stained Glass Wallpaper Seamless: Abstract seamless 

In [10]:
# ✅ 載入模型

FastVisionModel.for_inference(model) # Enable for inference!
model.eval().to("cuda")


# ✅ 載入 test 資料
test_df = pd.read_parquet("test_data.parquet")
test_dataset = Dataset.from_pandas(test_df)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]

# ✅ bytes ➜ PIL image
def convert_bytes_to_pil(image_bytes):
    return Image.open(BytesIO(image_bytes)).convert("RGB")

def clean_caption(text):
    # 移除特殊 token
    text = re.sub(r"<\|.*?\|>", "", text)

    # 移除角色名與 user prompt
    text = re.sub(r"user\s*\n*", "", text, flags=re.IGNORECASE)

    # 移除 prompt 本身（防止模型複述）
    prompt = "You are an expert radiographer. Describe accurately what you see in this image."
    text = text.replace(prompt, "")
    text = text.replace("assistant", "")
    text = re.sub(r"\n+", " ", text)

    return text.strip()

# ✅ 推理函數（單張圖）
def predict_caption(image_bytes):
    image = convert_bytes_to_pil(image_bytes)
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")
    text_streamer = TextStreamer(tokenizer, skip_prompt = True)
    outputs = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    if "<|assistant|>" in decoded:
        caption = decoded.split("<|assistant|>")[-1].strip()
    else:
        caption = decoded.strip()

    # 清除任何殘留特殊 token
    return clean_caption(caption)


# ✅ 推理全部 test data
predictions = []
for row in tqdm(test_dataset, total=len(test_dataset)):
    image_bytes = row["image"]["bytes"]
    caption = predict_caption(image_bytes)
    predictions.append(caption)

# ✅ 儲存預測結果
test_df["caption"] = predictions
test_df[["idx", "caption"]].to_csv("test_predictions.csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

Christmas Bokeh Lights Background: Yellow Lights Bokeh Background for Christmas and New Year Celebration<|eot_id|>


  1%|          | 1/100 [00:30<49:55, 30.26s/it]

Zion Park - Zion National Park, Utah, United States<|eot_id|>


  2%|▏         | 2/100 [00:35<25:22, 15.54s/it]

...

 98%|█████████▊| 98/100 [09:19<00:10,  5.24s/it]

Christmas Tree Window Art: Digital Art Illustration of Christmas Tree Window Art<|eot_id|>


 99%|█████████▉| 99/100 [09:24<00:05,  5.13s/it]

Holly Paper: Christmas wrapping paper: holly with berries and leaves on cream background<|eot_id|>


100%|██████████| 100/100 [09:30<00:00,  5.70s/it]


In [11]:
import pandas as pd
import json

# 讀入 CSV 檔案
df = pd.read_csv("test_predictions.csv")

# 將 DataFrame 轉為符合格式的 JSON 結構
results = [{"idx": int(row["idx"]), "output": row["caption"]} for _, row in df.iterrows()]

# 儲存為 JSON 檔案（例如 output_submission.json）
with open("submission.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("✅ 已成功轉換為 submission.json 格式")


✅ 已成功轉換為 submission.json 格式


In [12]:
from google.colab import files
files.download("submission.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

模型設定：

Vision 層微調: finetune_vision_layers = True

語言層微調: finetune_language_layers = True

注意力層微調: finetune_attention_modules = True

MLP 層微調: finetune_mlp_modules = True

LoRA: r = 32, lora_alpha = 32，lora_dropout = 0.05

其他: use_rslora = True

訓練批次大小: per_device_train_batch_size = 2

梯度累積步數: gradient_accumulation_steps = 8

學習率: learning_rate = 2e-4

優化器: 使用 "adamw_8bit"

學習率調度: "cosine"。

最大步數: max_steps = 30



我的code啟用了 finetune_vision_layers = True，對語言層進行微調，還對視覺層進行微調。這對於多模態模型來說是重要的，尤其是在圖像和文本互動的任務中，視覺層的微調能幫助模型更好地理解和生成視覺內容。

LoRA配置：

使用了更大的 r  和 lora_alpha，以及稍微提高的 lora_dropout 。這些變更有助於增加模型的表達能力，可能會提升性能，但同時也增加了過擬合的風險。use_rslora = True 這個設置可能有助於降低過擬合，並提高模型在不同數據集上的穩定性。

訓練設定的變化：

增加了 gradient_accumulation_steps = 8，這意味著每次更新模型參數時會累積更多步驟，從而減少顯存的需求並且能夠處理更大的批次。

調整了學習率調度方式，由原本的 linear 調度變為 cosine 調度。這樣的學習率衰減有助於在訓練後期更穩定地收斂，避免訓練過程中的劇烈變化。